# Voice Symptom Intake & Documentation Assistant - Google Colab Deployment

This notebook deploys the Voice Symptom Intake & Documentation Assistant on Google Colab with GPU support.

## ✅ Advantages of Colab Deployment:
- **Free GPU Access** (Tesla T4 with 16GB VRAM)
- **No Local Setup Issues** (no Windows file locking, FFmpeg, etc.)
- **Faster Inference** (MedASR & MedGemma both run on GPU)
- **Public URL Access** via ngrok

## ⚠️ Important Notes:
- Sessions last up to 12 hours (free tier)
- You'll need a **Hugging Face token** with "Read" access
- Accept model terms for `google/medasr` and `google/medgemma-1.5-4b-it`

## Step 1: Check GPU Availability

In [ ]:
!nvidia-smi

## Step 2: Install Dependencies

Install all required packages (this takes ~3-5 minutes)

In [ ]:
%%capture
# Install transformers from specific commit for MedASR support
!pip install git+https://github.com/huggingface/transformers.git@65dc261512cbdb1ee72b88ae5b222f2605aad8e5

# Install other dependencies
!pip install fastapi uvicorn[standard] python-multipart
!pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install accelerate librosa soundfile noisereduce audioread
!pip install pydantic pydantic-settings python-dotenv
!pip install pyngrok nest-asyncio

print("✅ All dependencies installed successfully!")

## Step 3: Upload Your Code

**Option A:** Upload project folder from your computer
- Click the folder icon on the left sidebar
- Upload the entire `voice-symptom-triage-assistant` folder

**Option B:** Clone from GitHub (if you've pushed your code)

In [ ]:
# Option B: Clone from GitHub (uncomment and modify if using)
# !git clone https://github.com/YOUR_USERNAME/voice-symptom-triage-assistant.git
# %cd voice-symptom-triage-assistant

# Option A: If uploaded manually, navigate to the folder
%cd /content/voice-symptom-triage-assistant

## Step 4: Configure Hugging Face Token

**Get your token from:** https://huggingface.co/settings/tokens

**Make sure you've accepted terms for:**
- https://huggingface.co/google/medasr
- https://huggingface.co/google/medgemma-1.5-4b-it

In [ ]:
import os

# REPLACE WITH YOUR HUGGING FACE TOKEN
HF_TOKEN = "hf_YOUR_TOKEN_HERE"

# Create .env file
with open('.env', 'w') as f:
    f.write(f"HF_TOKEN={HF_TOKEN}\n")
    f.write("MEDASR_MODEL=google/medasr\n")
    f.write("MEDGEMMA_MODEL=google/medgemma-1.5-4b-it\n")
    f.write("DEVICE=cuda\n")
    f.write("ENABLE_GPU=true\n")
    f.write("AUDIO_SAMPLE_RATE=16000\n")
    f.write("MAX_AUDIO_DURATION_SECONDS=300\n")
    f.write("MODEL_CACHE_DIR=./models\n")
    f.write("LOG_LEVEL=INFO\n")

print("✅ Environment configured!")

## Step 5: Update MedGemma for GPU (Colab has 16GB VRAM!)

Since Colab has more VRAM than GTX 1650, we can run MedGemma on GPU

In [ ]:
# Update medgemma_service.py to use GPU
medgemma_fix = '''
            # Colab GPU has 16GB VRAM - can fit MedGemma!
            dtype = torch.float16 if torch.cuda.is_available() else torch.float32
            
            self.tokenizer = AutoTokenizer.from_pretrained(
                settings.medgemma_model,
                token=settings.hf_token if settings.hf_token else None
            )
            
            self.model = AutoModelForCausalLM.from_pretrained(
                settings.medgemma_model,
                torch_dtype=dtype,
                device_map="auto" if settings.enable_gpu else None,
                token=settings.hf_token if settings.hf_token else None
            )
            
            self.model.eval()
'''

print("Note: MedGemma will use GPU on Colab (16GB VRAM available)")

## Step 6: Set Up ngrok Tunnel

**Get your ngrok authtoken from:** https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
from pyngrok import ngrok
import nest_asyncio

# REPLACE WITH YOUR NGROK AUTHTOKEN
NGROK_AUTH_TOKEN = "YOUR_NGROK_TOKEN_HERE"

# Set up ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
nest_asyncio.apply()

print("✅ ngrok configured!")

## Step 7: Start the Server

This will:
1. Load MedASR model (~30 seconds)
2. Load MedGemma model (~2-3 minutes first time)
3. Start the FastAPI server
4. Create a public ngrok URL

In [ ]:
import subprocess
import threading
import time

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"\n{'='*60}")
print(f"🌐 PUBLIC URL: {public_url}")
print(f"{'='*60}\n")
print("Open this URL in your browser to access the application!\n")

# Start uvicorn server
!python -m uvicorn app.main:app --host 0.0.0.0 --port 8000

## Step 8: Access Your Application

1. **Copy the public ngrok URL** from the output above
2. **Open it in your browser**
3. **Start recording** or **upload audio files**
4. **View transcription and documentation results**

## ✅ Features:
- Audio recording directly in browser
- File upload (WAV, MP3, M4A, FLAC, OGG)
- Real-time transcription with MedASR
- Structured documentation with MedGemma
- Export results as JSON
- Copy to clipboard

## 🛑 To Stop:
- Click the **Stop** button in Colab
- Or press **Ctrl+C** in the cell output

## 📝 Notes:
- Free Colab sessions last **up to 12 hours**
- The ngrok URL **changes each time** you restart
- Models are **cached** after first download (faster restarts)
- GPU inference is **10x faster** than CPU

## Troubleshooting

### If models fail to load:
1. Check your HF_TOKEN is valid
2. Verify you accepted model terms on Hugging Face
3. Try restarting the runtime: Runtime → Restart runtime

### If ngrok fails:
1. Verify your ngrok authtoken
2. Free ngrok accounts have limits (1 tunnel at a time)
3. Try getting a new authtoken from ngrok dashboard

### If audio fails:
1. Check browser microphone permissions
2. Hard refresh browser (Ctrl+Shift+R)
3. Ensure audio files are under 5 minutes (default limit)